In [2]:
import pandas as pd
import numpy as np
import os
import sys
import time

import string
import json
import pickle as pk

from collections import Counter

from math import sin, asin, cos, radians, fabs, sqrt

# POI_list     
# st_p   停留点列表   三元组list    [lat, lng, time]
# st_poi  停留点-POI列表，   四元组list   [id, lat, lng, time] 
# st_poi_dic    停留点-POI字典     字典list   [{id, location, staying,category},{},{}]
# st_poi_tree   停留点-POI树     字典树   根节点是字典，大类是字典，小类是list


In [3]:
############################    从文件中 读取 数据   ##############
def read_q(filename):
    f = open(filename,"rb+")
    st = pk.load(f)
    f.close()
    return st

In [4]:
##########################    向文件中  写 数据  #####################
def write_q(st, filename):
    f = open(filename, "wb+")
    pk.dump(st,f)
    f.close()

In [61]:
####################   proba()  把 staying list  转换为概率特征    ############
#  输入: staying list   一个POI上的停留时间list ;  把停留时间化为了9个时段0::8
#  输出: st_f      停留时间概率分布，  
def proba(st):
    
    t = st.copy()
    for i in range(0, len(st)):
    
        if 10 <= st[i] < 30:
            t[i] = 0
        elif 30 <= st[i] < 60:
            t[i] = 1
        elif 60 <= st[i] < 90:
            t[i] = 2
        elif 90 <= st[i] < 120:
            t[i] = 3
        elif 120 <= st[i] < 150:
            t[i] = 4
        elif 150 <= st[i] < 180:
            t[i] = 5
        elif 180 <= st[i] < 210:
            t[i] = 6
        elif 210 <= st[i] < 240:
            t[i] = 7
        else:
            t[i] = 8
    
    b = len(t)
    a = Counter(t)      #  ？？？？？
    #print(a)
    stf = [0]*9        # staying list 转化为 fecture vector 【0,0,0，，，0】
    #print(st_f)

    for key, value in a.items():
        stf[key] = round(value/b, 3)
        
    return stf   

In [62]:
################################  停留点 ----> 转换轨迹段 到 停留点  #############
# 计算停留点 stay_point 的平均地理位置 及 停留时间
# input:  一个 stay_point 轨迹段，       所有位置 及 访问时刻  [[Lat, Long, Time],[],[]]
# output:  这个stay_point 的平均地理位置 及 停留时长 [Lat, Long, Duration]   单位， 分钟/min
def mean_point(a):
    b = 0
    c = 0
    for i in range(0, len(a)):
        b = b + a[i][0]
        c = c + a[i][1]
    
    b = b / len(a)
    c = c / len(a)
    
    d = (a[len(a)-1][2] -a[0][2]) * 24*60 
    
    return [b, c, d]

In [63]:
###################################   单停留点与 POI的匹配    ############
# 输入:  停留点list (一个 三元组)[经度，纬度，时间] , POI_list  [“名称”，纬度，经度]
# 输出： 最近的POI  及 停留时间
#st_p = st_p_list[0]

def match(st_p, POI_list):
    poi_st_1 = [] 
    for i in range(0, len(POI_list)):
        a = [[POI_list[i][1], POI_list[i][2]],[st_p[0],st_p[1]]]
        b = pd.DataFrame(a)
        c = Dis(b)
        poi_st_1.append(c)    
    #print(len(poi_st_1)) 
    j = poi_st_1.index( min(poi_st_1) )
    return  [POI_list[j][0],POI_list[j][1], POI_list[j][2], st_p[2]]      #【“名称”，纬度，经度，停留时间】
    

In [64]:
#######################   Dis()  根据经纬度计算距离 distance(a)  ####################3
#     input:  a  2*2 两个点的经纬度坐标；
#     output:  距离  m

################    测试用例   #################
#a = [[40.009532,116.322153],[40.009493,116.322103]]
#b = pd.DataFrame(a)
#Dis(b)

EARTH_RADIUS = 6371.004           #地球平均半径，6371km
 
def hav(theta):
    s = sin(theta / 2)
    return s * s
 
def Dis(a): 
#def get_distance_hav(lat0, lng0, lat1, lng1):
    """用haversine公式计算球面两点间的距离。"""
    # 经纬度转换成弧度
    lat0 = radians(a.iloc[0,0])
    lat1 = radians(a.iloc[1,0])
    lng0 = radians(a.iloc[0,1])
    lng1 = radians(a.iloc[1,1])
 
    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    dist = 2 * EARTH_RADIUS * asin(sqrt(h))*1000
 
    return dist

In [20]:
##############################   POI_list  写   ###########################
file_poi_list = ".\\os_file\\POI_list.pkl"
write_q(POI_list, file_poi_list)

In [69]:
##############################   POI_list  读   ###########################
file_poi_list = ".\\os_file\\POI_list.pkl"
POI_list = read_q(file_poi_list)

In [70]:
len(POI_list)

76061

In [22]:
####################################   读 st_p_1/2 轨迹  ###############################
# 输出 st_p_1, st_p_2  为原始停留点数据-轨迹   四重list

file_name_st1 = ".\\Stay_point\\st_p_1-20190726-120655.txt"        # 读取第一类停留点
b = open(file_name_st1, "r+")
out = b.read()
st_p_1 =  json.loads(out)
b.close()

file_name_st2 = ".\\Stay_point\\st_p_2-20190726-120655.txt"        # 读取第二类停留点
b = open(file_name_st2, "r+")
out = b.read()
st_p_2 =  json.loads(out)
b.close()

len(st_p_2[0])

In [23]:
###########################  形成 st_p list  三元组list #################################################
# st_p_1/2   从TXT文件中，直接读取的数据       四重list st_p[i][j][k][m]    i user;   j file;   k st_p;  m  GPS_point
# st_p        停留点list      三元组list       [lat, lng, time]    

filename = ".\\os_file\\st_p.pkl"
st_p = []    #  停留点的list 

st_p_m = st_p_1
for i in range(0, len(st_p_m)):        
             # 对 st_p_m[i]  第i个用户，操作文件  user_files
    if len(st_p_m[i]) == 0:            #  如果  user_file  的文件数为 0，  len(st_p_1[i])是用户的文件数
        continue
    else:       
        for j in range(0, len(st_p_m[i])):      
                    # 对用户i的文件j, st_p[i][j],     文件
            if len(st_p_m[i][j]) == 0:       #  文件内没有stay point，
                continue
            else:      #  文件内有 stay point               #  user_file_st-p
                
                for k in range(0, len(st_p_m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                    #st_p_1.append(st_p_1_txt[i][j][k])
                    if len(st_p_m[i][j][k]) ==0:
                        continue
                    else:                # 操作每一个 st_p_1_txt[i][j][k] 停留点
                        
                        #此处定义一个 POI
                        a = mean_point(st_p_m[i][j][k])
                        #print(a)
                        st_p.append(a)
                            
b = len(st_p)
print("第一类停留点有： ", b,"  个")

st_p_m = st_p_2
for i in range(0, len(st_p_m)):        
             # 对 st_p[i]  第i-1个用户，操作文件  user_files
    if len(st_p_m[i]) == 0:            #  如果  user_file  的文件数为 0，  len(st_p_1[i])是用户的文件数
        continue
    else:       
        for j in range(0, len(st_p_m[i])):      
                    # 对用户i的文件j, st_p[i][j],  操作停留点  user_file_st-p
            if len(st_p_m[i][j]) == 0:       #  文件内没有stay point，
                continue
            else:      #  文件内有 stay point               #  user_file_st-p
                
                for k in range(0, len(st_p_m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]   操作地理点  user_file_st-p_GPS-point
                    #st_p_1.append(st_p_1_txt[i][j][k])
                    if len(st_p_m[i][j][k]) ==0:
                        continue
                    else:                # 操作每一个 st_p_1_txt[i][j][k] 停留点
                        
                        #此处定义一个 POI
                        a = mean_point(st_p_m[i][j][k])
                        #print(a)
                        st_p.append(a)                            
c = len(st_p)
print("第二类停留点有： ", c - b,"  个")
print("共有停留点有： ", c,"  个")
#len(st_p_list)    #  三元组   [lat,lng, time]

write_q(st_p, filename)
#filename1 = ".\\os_file\\st_p.csv"
#write_st_p(st_p, filename1)


第一类停留点有：  2870   个
第二类停留点有：  15471   个
共有停留点有：  18341   个


In [66]:
#############################    读 st_p   ##########################
filename = ".\\os_file\\st_p.pkl"
st_p = read_q(filename)
len(st_p)

18341

In [75]:
len(st_p)

18341

In [76]:
################################   st_poi   停留点 与 POI 匹配  #####################################
#  停留点   st_p
#  POI      POI_list
#  输出  st_poi      停留点对应的POI，及停留时间  [id, lat, lng, time]  
filename = ".\\os_file\\st_poi.pkl"   
st_poi = []
#for i in range(0, len(st_p)):
for i in range(0, 20):
    
    st_p_m = match(st_p[i], POI_list)
    st_poi.append(st_p_m)
    
write_q(st_poi, filename)

In [72]:
st_poi

[['圆明园-心境轩', 40.006578000000005, 116.320843, 23.749999915016815],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 212.31666662497446],
 ['科苑公寓', 39.992231, 116.327031, 35.83333340124227],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 180.0],
 ['人保博物馆', 40.007597, 116.320035, 59.0666667802725],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 21.683333372930065],
 ['DQ(双榆树店)', 39.973544, 116.332475, 27.699999979231507],
 ['无尽', 40.005973, 116.32533400000001, 68.36666659684852],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 99.79999992297962],
 ['无尽', 40.005973, 116.32533400000001, 113.54999989969656],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 172.9333332006354],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 27.00000000419095],
 ['无尽', 40.005973, 116.32533400000001, 63.416666738921776],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 196.2333332723938],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 82.84999997355044],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 50.7666666212026],
 ['人保博物馆', 40.007597, 116.320035, 44.59999996935949],
 ['无尽', 

In [27]:
###############################   读 st_poi   ###########################
filename = ".\\os_file\\st_poi.pkl"
st_poi = read_q(filename)
st_poi

[['圆明园-心境轩', 40.006578000000005, 116.320843, 23.749999915016815],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 212.31666662497446],
 ['科苑公寓', 39.992231, 116.327031, 35.83333340124227],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 180.0],
 ['人保博物馆', 40.007597, 116.320035, 59.0666667802725],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 21.683333372930065],
 ['DQ(双榆树店)', 39.973544, 116.332475, 27.699999979231507],
 ['无尽', 40.005973, 116.32533400000001, 68.36666659684852],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 99.79999992297962],
 ['无尽', 40.005973, 116.32533400000001, 113.54999989969656]]

In [73]:
st_poi

[['圆明园-心境轩', 40.006578000000005, 116.320843, 23.749999915016815],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 212.31666662497446],
 ['科苑公寓', 39.992231, 116.327031, 35.83333340124227],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 180.0],
 ['人保博物馆', 40.007597, 116.320035, 59.0666667802725],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 21.683333372930065],
 ['DQ(双榆树店)', 39.973544, 116.332475, 27.699999979231507],
 ['无尽', 40.005973, 116.32533400000001, 68.36666659684852],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 99.79999992297962],
 ['无尽', 40.005973, 116.32533400000001, 113.54999989969656],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 172.9333332006354],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 27.00000000419095],
 ['无尽', 40.005973, 116.32533400000001, 63.416666738921776],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 196.2333332723938],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 82.84999997355044],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 50.7666666212026],
 ['人保博物馆', 40.007597, 116.320035, 44.59999996935949],
 ['无尽', 

In [17]:
#############################  st_poi_dic   把 st_p 四元组list 转换成 字典列表  ###################### 
#  根据POI 【id, lat, lng】 对停留时间进行聚集。  
#  输出 st_p_dic  是 字典list        st_p_dic[i][][]    第i个POI，[][]字典的操作
#    POI  {'id': '无尽',
#          'location': {'lat': 40.005973, 'lng': 116.32533400000001},
#          'staying': [68.36666659684852, 113.54999989969656]}
filename = ".\\os_file\\st_poi_dic.pkl"
st_poi_dic = []

for i in range(0,len(st_poi)):
    num = 0
    for j in range(0, len(st_poi_dic) ):        # 遍历已有列表
        
        if  st_poi[i][0] == st_poi_dic[j]["id"] and st_poi[i][1] == st_poi_dic[j]["location"]["lat"]  and  \
        st_poi[i][2] == st_poi_dic[j]["location"]["lng"]:       # 如果列表中已有该字典条目
            
            st_poi_dic[j]["staying"].append( st_poi[i][3])
            break
        else:
            num += 1
            continue
    if num == len(st_poi_dic):        # 如果 现有st_poi_dic  list  中没有，则为该POI 添加 一个新的字典条目
        a = {}
        a["id"] = st_poi[i][0]
        a["location"] = {"lat": st_poi[i][1], "lng": st_poi[i][2]}
        a["staying"] = [st_poi[i][3]]
        
        st_poi_dic.append(a)
    else:
        continue

write_q(st_poi_dic, filename)

In [6]:
filename = ".\\os_file\\st_poi_dic.pkl"

st_poi_dic = read_q(filename)
len(st_poi_dic)

3634

In [19]:
#################################   st_poi_dic 分类    标签   ##########################
# 给 st_poi_dic 加语义标签，category         其本身是由字典组成的列表
# st_poi_dic =       
#   [{'id': 'DQ(双榆树店)',
#     'location': {'lat': 39.973544, 'lng': 116.332475},
#     'staying': [27.699999979231507]},
#    {'id': '无尽',
#     'location': {'lat': 40.005973, 'lng': 116.32533400000001},
#      'staying': [68.36666659684852, 113.54999989969656]}]

filename = ".\\os_file\\st_poi_label.pkl"
path = ".\\POI_base\\POI\\"             # 根目录
class1_list = os.listdir(path)           # 大类列表
for i in range(0, len(class1_list)):
    #print("当前大类： ", class1_list[i])        
    class2_path = path + class1_list[i]      # 大类路径 ， 每个大类
    class2_list = os.listdir(class2_path)     # 小类列表
    
    for j in range(0, len(class2_list)):
        class2_file = class2_path +"\\"+ class2_list[j]    # 小类路径 ， 每个小类
        #print("大类： ",class1_list[i],"下的当前小类： ",  class2_list[j])
        #print("小类文件目录： ", class2_file)
        
        f = open(class2_file,"rb")
        Lat_lng = pd.read_csv(f,header = None, encoding = "gbk")    # read.csv 后是DataFrame格式
        f.close()
        #print("小类poi数量 len(Lat_lng): ", len(Lat_lng))
 
        poi_list = []            # 暂存小类POI_list
        for r in range(0, len(Lat_lng)):
            a = [Lat_lng.iloc[r][0], Lat_lng.iloc[r][1], Lat_lng.iloc[r][2]]
            poi_list.append(a)
        
        #poi_list = read_POI(class2_file)         #[id, lat, lng]
        
        name = os.path.splitext(class2_list[j])[0]      # 取出小类名
        #base=os.path.basename(filename)    取文件全名  
        
        for k in range(0, len(poi_list)):
            
            for m in range(0, len(st_poi_dic)):
                
                if  poi_list[k][0] == st_poi_dic[m]["id"] and poi_list[k][1] == st_poi_dic[m]["location"]["lat"]  and  \
                poi_list[k][2] == st_poi_dic[m]["location"]["lng"]:       # 如果列表中已有该字典条目
            
                    st_poi_dic[m]["category"] = name
                    break

write_q(st_poi_dic, filename)

In [5]:
filename = ".\\os_file\\st_poi_dic.pkl"
st_poi_dic = read_q(filename)       # label 后
len(st_poi_dic)

3634

In [6]:
#################################   基于字典 建立 st_poi_tree 分类树   ##########################
"""#  大类 是 字典，  enter["scenic"]； 
#  小类 是 列表，  scenic[{},{},{}];   scenic[2]["staying"]
#  st_poi  是  字典， st_poi = {id: , location: , staying: , category: }"""

filename = ".\\os_file\\st_poi_tree.pkl"

path = ".\\POI_base\\POI\\"                               # 根目录
class1_list = os.listdir(path)                            # 大类列表
createVar = locals()    # 动态生成变量

st_poi_tree = {}
#print("所有大类： ", class1_list)  
for i in range(0, len(class1_list)):
    #print("当前大类： ", class1_list[i])        
    class2_path = path + class1_list[i]                   # 大类路径 ， 每个大类
    class2_list = os.listdir(class2_path)                 # 小类列表
    #print("小类列表： ", class2_list) 
    
    name1 = os.path.splitext(class1_list[i])[0]      # 取出大类名
    createVar[name1] = {}   # 创建大类字典
    
    for j in range(0, len(class2_list)):
        class2_file = class2_path +"\\"+ class2_list[j]     # 小类路径 ， 每个小类
        #print("大类： ",class1_list[i],"下的当前小类： ",  class2_list[j])
        #print("小类文件目录： ", class2_file)
        name2 = os.path.splitext(class2_list[j])[0]      # 取出小类名
        createVar[name2] = []      #创建小类  列表
        
        
        for k in range(0, len(st_poi_dic)):         # 对st_poi_dic 进行分类
            
            if st_poi_dic[k]["category"] == name2:
                createVar[name2].append(st_poi_dic[k])
        
        
        
        createVar[name1][name2] = createVar[name2]    # 把小类字典 嵌入 大类字典
        
    st_poi_tree[name1] = createVar[name1]
        
write_q(st_poi_tree, filename)


In [7]:
###########################  读 st_poi_tree  #############
filename = ".\\os_file\\st_poi_tree.pkl"
st_poi_tree = read_q(filename)
#st_poi_tree["enter"]["scenic"][2]["staying"]
#st_poi_tree


In [9]:
st_poi_tree["enter"]["bar"]

[{'id': '大堂吧',
  'location': {'lat': 39.996445, 'lng': 116.312735},
  'staying': [77.04999993788078],
  'category': 'bar'},
 {'id': '京都信苑饭店-大堂酒吧',
  'location': {'lat': 39.904334999999996, 'lng': 116.31858600000001},
  'staying': [12.249999940395355],
  'category': 'bar'},
 {'id': '半山酒吧',
  'location': {'lat': 39.973151, 'lng': 116.48791299999999},
  'staying': [35.41666665812954, 151.58333333325572],
  'category': 'bar'},
 {'id': 'juice bar',
  'location': {'lat': 39.918248, 'lng': 116.31093600000001},
  'staying': [42.083333290647715, 151.28333336790092],
  'category': 'bar'},
 {'id': '酒吧',
  'location': {'lat': 39.975998, 'lng': 116.32942800000001},
  'staying': [14.19999997713603,
   20.000000012805685,
   20.016666578594595,
   14.400000002933666,
   10.516666610492393,
   12.366666707675904,
   46.233333349227905,
   20.39999990724027,
   14.499999932013452,
   14.466666674707085,
   109.61666668416001,
   90.96666666213423,
   141.46666675340384,
   31.41666661365889,
   48.0166

In [50]:
####################    K means  数据准备 samples[][]  两重list  ############################
# 数据集 samples  底层 每个POI staying_list 转化为概率分布特征向量  st_f = proba(st)
# 0: [10,30), 1:[30,60),...8:[240,..);     st_f  为staying list 内，各时段停留出现的频率
#
"""st_poi_tree ---> samples """

#st_poi_tree
filename = ".\\os_file\\samples.pkl"
samples = []      #  samples[0][6]    # 大类 enter 中的小类 scenic 的样本数据 list
for item1 in st_poi_tree:
    
    a1 = []
    for item2 in st_poi_tree[item1]:        #  st_poi_tree["enter"]["bar"][list]
        
        #print(item1 ,"  ---> ",  item2)
        #print(len(st_poi_tree[item1][item2]))    #
        a2 = []
        for i in range(0, len(st_poi_tree[item1][item2])):  #  
            
            a3 = st_poi_tree[item1][item2][i]["staying"]
            
            a4 = proba(a3)       # proba()  把staying list 转化为 各时段的概率分布（4小时）
            a2.append(a4)
        a1.append(a2)
    samples.append(a1)
        
#samples
write_q(samples, filename)

In [52]:
print(st_poi_tree["enter"]["scenic"])
samples[0][6]    # 大类 enter 中的小类 scenic 的样本数据 


[{'id': '圆明园-心境轩', 'location': {'lat': 40.006578000000005, 'lng': 116.320843}, 'staying': [23.749999915016815], 'category': 'scenic'}, {'id': '人保博物馆', 'location': {'lat': 40.007597, 'lng': 116.320035}, 'staying': [59.0666667802725], 'category': 'scenic'}, {'id': '无尽', 'location': {'lat': 40.005973, 'lng': 116.32533400000001}, 'staying': [68.36666659684852, 113.54999989969656], 'category': 'scenic'}]


[[1.0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1.0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0.5, 0.5, 0, 0, 0, 0, 0]]

In [54]:
#########################   K means 聚类   ##########################
# 数据集 sample  两重list samples中的一个数据集  sample[0][6]  第大类第7小类： enter --> scenic 
#
#

a  = np.array(samples[0][6])




array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. ]])

In [ ]:
#################################   删除异常点   ##########################
path = ".\\POI_base\\POI\\"             # 根目录
class1_list = os.listdir(path)           # 大类列表
for i in range(0, len(class1_list)):
    #print("当前大类： ", class1_list[i])        
    class2_path = path + class1_list[i]      # 大类路径 ， 每个大类
    class2_list = os.listdir(class2_path)     # 小类列表
    
    for j in range(0, len(class2_list)):
        class2_file = class2_path +"\\"+ class2_list[j]    # 小类路径 ， 每个小类
        #print("大类： ",class1_list[i],"下的当前小类： ",  class2_list[j])
        #print("小类文件目录： ", class2_file)
        
        f = open(class2_file,"rb")
        Lat_lng = pd.read_csv(f,header = None, encoding = "gbk")    # read.csv 后是DataFrame格式
        f.close()
        #print("小类poi数量 len(Lat_lng): ", len(Lat_lng))
        
        for k in range(0, range(Lat_lng)):
            float(Lat_lng.iloc[k][1])
            float(Lat_lng.iloc[k][2])

In [ ]:
def read_POI(filename): 
    #filename = ".\\area_segment.csv"       #  读取搜索区域
    f = open(filename,"rb")
    Lat_lng = pd.read_csv(f,header = None, encoding = "gbk")
    f.close()

    POI_list = []
    for k in range(0,len(Lat_lng)):
        a = [Lat_lng.iloc[k][0],float(Lat_lng.iloc[k][1]),float(Lat_lng.iloc[k][2])]
        POI_list.append(a)
    return POI_list

In [ ]:
def write_st_poi(st_p, filename):        # st_p 是 与停留点匹配后的POI   
    f = open(filename, 'w+')
    for i in range(0, len(st_p)):
    
        a = st_p[i][0] + "," + str(st_p[i][1]) + "," + str(st_p[i][2]) +","+ str(st_p[i][3])
        f.write(a)
        f.write("\n")
    
    f.close()

In [ ]:
def write_st_p(st_p, filename):    #st_p_list   是提取出来的所有的停留点
    f = open(filename, 'w+')
    for i in range(0, len(st_p)):
    
        a = str(st_p[i][0]) + "," + str(st_p[i][1]) +","+ str(st_p[i][2])
        f.write(a)
        f.write("\n")
    
    f.close()

In [ ]:
#######################  读 POI_list  从文件中  ######################
file_poi_list = ".\\os_file\\POI_list.csv"


    #filename = ".\\area_segment.csv"       #  读取搜索区域
f = open(file_poi_list, "rb")
Lat_lng = pd.read_csv(f,header = None, encoding = "gbk")
f.close()

POI_list = []
for k in range(0,len(Lat_lng)):
    a = [Lat_lng.iloc[k][0],float(Lat_lng.iloc[k][1]),float(Lat_lng.iloc[k][2])]
    POI_list.append(a)
    
len(POI_list)

In [ ]:
#############################    读 st_p   ##########################
def read_st_p(filename):
    
    f = open(filename, "r+")
    st_p_m = pd.read_csv(f,header = None)
    f.close()
    
    st_p = []
    for i in range(0, len(st_p_m)):
        a = [st_p_m.iloc[i][0], st_p_m.iloc[i][1], st_p_m.iloc[i][2]]
        st_p.append(a)
    
    return st_p